In [3]:
import json
import numpy as np
import os
import re
import nltk
from nltk.corpus import gutenberg

# Making language model:

In [109]:
from nltk.util import ngrams
import string
from nltk import SimpleGoodTuringProbDist
from math import log
import random
from nltk.corpus import twitter_samples

In [41]:
 # rU means "read", and handles line endings
file= open("sample.txt", "rU")
raw= file.read()
print(raw[:11])
tokens= nltk.word_tokenize(raw)
#sample = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "sample.txt")
#test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
print(tokens[:30])
text= nltk.Text(tokens)
print(text[:20])
file.close()

C:\Users\Sudhanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


 Chef salad
['Chef', 'salad', 'is', 'calling', 'my', 'name', 'Im', 'so', 'hungry', 'Ngapain', 'lo', 'online', 'sih', 'nif', 'Iyaaaa', 'gituuu', 'oooooh', 'gituuuuu', 'haha', 'Im', 'on', 'my', 'iPod', 'and', 'Im', 'on', 'the', 'tumblr', 'app', 'and']
['Chef', 'salad', 'is', 'calling', 'my', 'name', 'Im', 'so', 'hungry', 'Ngapain', 'lo', 'online', 'sih', 'nif', 'Iyaaaa', 'gituuu', 'oooooh', 'gituuuuu', 'haha', 'Im']


In [132]:
tokens= nltk.corpus.gutenberg.words()
#sample = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "sample.txt")
#test = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "test_data.txt")
print(tokens[:30])
text= nltk.Text(tokens)
print(text[:20])
len(tokens)

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed']
['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich']


2621613

## Unigram perplexity with add 1 smoothing\ Laplacian smoothing

In [7]:
stopwords= set(nltk.corpus.stopwords.words('english'))
#stopwords

In [133]:
tokens[-20:]

['knob',
 'really',
 'undoing',
 ',',
 'turning',
 '--',
 'so',
 'now',
 'finally',
 ',',
 'Good',
 '-',
 'bye',
 '--',
 'and',
 'hail',
 '!',
 'my',
 'Fancy',
 '.']

In [135]:
unigram= [w.lower() for w in tokens if w not in stopwords]
unigram= list(set(unigram))
len(unigram)
#print(unigram[:10])

42334

In [136]:
spl= int(95*len(unigram)/100)
train_corpus= unigram[:spl]
test_corpus= unigram[spl:]

In [137]:
freq_1gram= nltk.FreqDist(train_corpus)
len_train= len(train_corpus)
vocab= len(set(train_corpus))

In [138]:
def unigram_prob_with_add1smoothing(word):
    return (freq_1gram[word]+1)/(len_train+vocab)

In [139]:
def entropy(n, text):
        e = 0.0
        text = ["<s>"] + text + ["</s>"]
        for i in range(n - 1, len(text)):
            context = text[i - n + 1:i]
            token = text[i]
            #print(str(context)+"    "+token)
            e += logprob(token, context)
        return e / float(len(text) - (n - 1))


def logprob(word, context):
    if len(context)==0:
        p=unigram_prob_with_add1smoothing(word)
    elif len(context)==1:
        p=bigram_prob_with_add1smoothing(context[0], word)
    else:
        p=trigram_prob_with_add1smoothing(context[0], context[1], word)
    return -p*log(p , 2)


def perplexity(n, text):
      return pow(2.0, entropy(n, text))

In [140]:
perplexity(1, test_corpus)

1.000140437941288

In [141]:
perplexity(1, train_corpus)

1.0002636496062822

## Bigram Perplexity with add 1 smoothing

In [142]:
bigram= [a for a in ngrams(unigram, 2, pad_left=True,pad_right=True,left_pad_symbol='<s>', right_pad_symbol="</s>")]
print(bigram[:25])
len(bigram)

[('<s>', 'gallowes'), ('gallowes', 'therin'), ('therin', 'circumvented'), ('circumvented', 'leuell'), ('leuell', 'himmalehs'), ('himmalehs', 'forsaking'), ('forsaking', 'mitchell'), ('mitchell', 'civil'), ('civil', 'purchases'), ('purchases', 'marcus'), ('marcus', 'purveyed'), ('purveyed', 'maritime'), ('maritime', 'vpspring'), ('vpspring', 'surging'), ('surging', 'palsied'), ('palsied', 'disinterested'), ('disinterested', 'arrowy'), ('arrowy', 'ignorant'), ('ignorant', 'plantings'), ('plantings', 'forth'), ('forth', 'plautus'), ('plautus', 'improbability'), ('improbability', 'indestructible'), ('indestructible', 'sneezes'), ('sneezes', 'dead')]


42335

In [143]:
cfreq_2gram = nltk.ConditionalFreqDist(bigram)
cprob_2gram = nltk.ConditionalProbDist(cfreq_2gram, nltk.MLEProbDist)
def bigram_prob_with_add1smoothing(word1, word2):
    cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
    return cprob_2gram_add1
#print(cfreq_2gram.tabulate(conditions= ['khalifa', 'romance', 'tourist'], samples=range(100) ))
print(cfreq_2gram['travel'].most_common(20))

[('madam', 1)]


In [52]:
perplexity(2, test_corpus)

1.0002393074652438

In [53]:
perplexity(2, train_corpus)

1.0002393311563496

In [54]:
def bigram_prob_with_gtsmoothing(word1, word2):
    return SimpleGoodTuringProbDist(word1, word2)

In [55]:
def bigram_prob_with_add1smoothing(word1, word2):
    cprob_2gram_add1=float((((1+cfreq_2gram[word1][word2])/(len(cfreq_2gram)+sum(cfreq_2gram[word1].values())))))
    return cprob_2gram_add1

In [56]:
#bigram perplexity with Good-Turing smoothing( or add 1 smoothing )
per_value_2gram= perplexity(2, test_corpus)
per_value_2gram

1.0002393074652438

In [144]:
trigram= [a for a in ngrams(unigram, 3, pad_left= True, pad_right=True, left_pad_symbol= "<s>", right_pad_symbol="</s>")]
len(trigram)

42336

In [145]:
trigrams_as_bigrams=[]
trigrams_as_bigrams.extend([((t[0],t[1]), t[2]) for t in trigram])
cfreq_3gram = nltk.ConditionalFreqDist(trigrams_as_bigrams)

In [146]:
print(trigrams_as_bigrams[:40])
cfreq_3gram[('lanchar', 'oshua')]

[(('<s>', '<s>'), 'gallowes'), (('<s>', 'gallowes'), 'therin'), (('gallowes', 'therin'), 'circumvented'), (('therin', 'circumvented'), 'leuell'), (('circumvented', 'leuell'), 'himmalehs'), (('leuell', 'himmalehs'), 'forsaking'), (('himmalehs', 'forsaking'), 'mitchell'), (('forsaking', 'mitchell'), 'civil'), (('mitchell', 'civil'), 'purchases'), (('civil', 'purchases'), 'marcus'), (('purchases', 'marcus'), 'purveyed'), (('marcus', 'purveyed'), 'maritime'), (('purveyed', 'maritime'), 'vpspring'), (('maritime', 'vpspring'), 'surging'), (('vpspring', 'surging'), 'palsied'), (('surging', 'palsied'), 'disinterested'), (('palsied', 'disinterested'), 'arrowy'), (('disinterested', 'arrowy'), 'ignorant'), (('arrowy', 'ignorant'), 'plantings'), (('ignorant', 'plantings'), 'forth'), (('plantings', 'forth'), 'plautus'), (('forth', 'plautus'), 'improbability'), (('plautus', 'improbability'), 'indestructible'), (('improbability', 'indestructible'), 'sneezes'), (('indestructible', 'sneezes'), 'dead'),

FreqDist()

In [147]:
cfreq_2gram['transpired']

FreqDist({'reclining': 1})

## Trying to generate text from corpus:

In [149]:
from collections import Counter, OrderedDict
from operator import itemgetter
import random

In [148]:
test_unigram = [w.strip(string.punctuation).lower() for w in test.words()]
test_unigram= [word for word in test_unigram if word not in stopwords]

test_bigram= [a for a in ngrams(test_unigram, 2)]
len(test_bigram)

test_trigram= [a for a in ngrams(test_unigram, 3)]
len(test_trigram)

test_frequency= nltk.FreqDist(test_bigram)
test_sorted_freq= OrderedDict(reversed(sorted(test_frequency.items(), key= itemgetter(1))))

test_cfreq_2gram= nltk.ConditionalFreqDist(test_bigram)

test_cprob_2gram= nltk.ConditionalProbDist(test_cfreq_2gram, nltk.MLEProbDist)

NameError: name 'test' is not defined

In [150]:
counts = Counter(tokens)
total_count = len(tokens)

In [151]:
counts.most_common(n=20)

[(',', 186091),
 ('the', 125748),
 ('and', 78846),
 ('.', 73746),
 ('of', 70078),
 (':', 47406),
 ('to', 46443),
 ('a', 32504),
 ('in', 31959),
 ('I', 30221),
 (';', 27329),
 ('that', 27289),
 ('he', 22198),
 ('his', 20585),
 ("'", 19873),
 ('it', 19734),
 ('was', 18558),
 ('for', 16860),
 ('not', 16834),
 ('with', 16827)]

In [152]:
for word in counts:
    counts[word] /= float(total_count)

In [153]:
# generating 100 words at random of the language
text=[]
for _ in range(100):
    r = random.random()
    accumulator = .0
 
    for word, freq in counts.items():
        accumulator += freq
 
        if accumulator >= r:
            text.append(word)
            break
print(' '.join(text))

throughout 19 him because Jesus in he she . , ? was : purpose well , dependence single which am said . . be any 2 thirty shalt had realm down , Moses nigh doors accursed , one shalt in her partial , , a 7 his 25 , , assured superior the delivered the for his a led as Nevertheless the , of drugged any to little in ; most then , did . the a the , of but betted , to and of was nor ." a Perceiving door once known top 9 the I pieces astonishment


## Random Text generate using bigram model

In [169]:
def text_generate_bigram(cfd_bigram, cfd_trigram, word, length=10):
    for i in range(length):
        word_list = nltk.word_tokenize(word)
        print(word, end=" ")
        if(len(word_list) == 1):
            word= cfd_bigram[word].max()
            #word= random.choice(list(cfreq_2gram[word].keys()))
        elif(len(word_list) == 2):
            word= cfd_trigram[(word_list[0], word_list[1])].max()
        else:
            print("Enter the entity no longer than 2 words!!!")
        #word= random.choice(list(cfreq_2gram[word].keys()))

In [170]:
text_generate_bigram(cfreq_2gram, cfreq_3gram, 'cricket')

cricket bruited anani disabling jot gaudiest shallows queene unanswered _engagement_ 

In [171]:
text_generate_bigram(cfreq_2gram,cfreq_3gram, "education")

education eliud darwin mungo azekah celibate girt ezri consonancy transgressors 

In [172]:
text_generate_bigram(cfreq_2gram,cfreq_3gram, "comedy")

comedy adornments leprous verified presumptive feverish forsookest forge armed collectively 

In [173]:
text_generate_bigram(cfreq_2gram,cfreq_3gram, "rich")

rich mazy federal excursive verbally evildoers reparation unpent dst misdoubt 

In [175]:
text_generate_bigram(cfreq_2gram,cfreq_3gram, "poor")

poor kirjath hatefullest trip vulcans translated hautboys crowns harass venome 

## POS template based random text generate

In [117]:
sent1= "Virat Kohli is a great cricket player of world"  #virat kohli
sent2= "Education gives us a knowledge of the world around us"  #education
sent3= "Rich is the new trend in the world"  #rich
sent4= "Comedy is a light way to tell the truth to people."  #comedy
sent5= "biggest tragedy here is that the accident could have easily been prevented"   #tragedy


for sentence in [sent1, sent2, sent3, sent4, sent5]:
    words= nltk.word_tokenize(sentence)
    print(nltk.pos_tag(words))

[('Virat', 'NNP'), ('Kohli', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('great', 'JJ'), ('cricket', 'NN'), ('player', 'NN'), ('of', 'IN'), ('world', 'NN')]
[('Education', 'NN'), ('gives', 'VBZ'), ('us', 'PRP'), ('a', 'DT'), ('knowledge', 'NN'), ('of', 'IN'), ('the', 'DT'), ('world', 'NN'), ('around', 'IN'), ('us', 'PRP')]
[('Rich', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('new', 'JJ'), ('trend', 'NN'), ('in', 'IN'), ('the', 'DT'), ('world', 'NN')]
[('Comedy', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('light', 'JJ'), ('way', 'NN'), ('to', 'TO'), ('tell', 'VB'), ('the', 'DT'), ('truth', 'NN'), ('to', 'TO'), ('people', 'NNS'), ('.', '.')]
[('biggest', 'JJS'), ('tragedy', 'NN'), ('here', 'RB'), ('is', 'VBZ'), ('that', 'IN'), ('the', 'DT'), ('accident', 'NN'), ('could', 'MD'), ('have', 'VB'), ('easily', 'RB'), ('been', 'VBN'), ('prevented', 'VBN')]


In [ ]:
template1= ['NN', 'VB' , 'JJ']